In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, ReLU, Add
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Covertype dataset
data = fetch_covtype()
X = data.data
y = (data.target == 2).astype(int)  # Binary classification: Class 2 vs. rest

# Normalizing features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Splitting into train/validation
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Select one batch (128 samples) to overfit
X_train_batch = X_train[:128]
y_train_batch = y_train[:128]

# Model Definition
input_dim = X_train.shape[1]
inputs = Input(shape=(input_dim,))

# First Dense Layer
x = Dense(32, activation='relu')(inputs)

# Residual Block
res = Dense(32, activation='relu')(x)
res = Dense(32)(res)
res = Add()([x, res])  # Residual connection
res = ReLU()(res)

# Additional Skip Connection (from input to deeper layer)
skip = Dense(32)(inputs)

# Combine residual output with skip connection
combined = Add()([res, skip])
combined = ReLU()(combined)

# Final dense layers
x = Dense(16, activation='relu')(combined)
outputs = Dense(1, activation='sigmoid')(x)

# Define model
model = Model(inputs=inputs, outputs=outputs)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Training on Single Batch 
history = model.fit(X_train_batch, y_train_batch, epochs=500, verbose=0)

# Validation Check 
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
train_loss, train_acc = model.evaluate(X_train_batch, y_train_batch, verbose=0)

# Save Model for Netron
model.save("/Users/melitamadhurza/Desktop/AIT_Deep_Learning/Notebooks/residual_mlp_model.h5")

# Print Required Info
print("Number of parameters:", model.count_params())
print("Final training loss on single batch:", train_loss)
print("Final validation loss:", val_loss)

Number of parameters: 6177
Final training loss on single batch: 4.469976829568623e-06
Final validation loss: 5.094172477722168
